In [3]:
# Ejecutando esta celda permito que las funciones
# que importe en este notebook se actualizen
# automaticamente si las cambio en los scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp2d


# Con este comando podre importar las funciones
# desde los scripts (asi trabajamos directamente
# en los scripts y evitamos tener una copia de cada
# funcion en el notebook)
import sys
sys.path.insert(0, '..')

from src.velocity import interpolate_discrete
from src.velocity import velocity_composition
from src.velocity import boat_movement

## Definitions

+ I call `boat_velocity` to the equivalent velocity of the boat if there were not streams. `boat_velocity` is a tuple for both the angle (`theta`, in radians) and the `velocity_mod` (in the same units as the `stream_velocities`)
+ I call `stream_velocity` to the velocity of the stream. 
+ I call `velocity` to the composition between both velocities

In [39]:
boat_velocity = np.array([10, 10])  # lat, lon
lat = 0
long = 3
vel_array = np.random.RandomState(seed=2).rand(10,20,2)
array_lat = np.arange(-5,5)
array_long = np.arange(-13,7)
ts = 0.1

velocity_composition(boat_velocity, lat, long, vel_array, array_lat, array_long)

array([10.33074703, 10.22861025])

In [40]:
boat_movement((np.array(11), np.array(3)), (1, 4.2), vel_array, xy_positions, ts)

TypeError: boat_movement() missing 2 required positional arguments: 'stream_velocities_longitudes' and 'ts'

In [33]:
np.zeros([2])

array([0., 0.])